In [1]:
import pandas as pd
import numpy as np
from data_master import DataMaster
from utils import func
from scipy.stats import norm
master = DataMaster()
from utils.func import center
from quantstats.stats import sharpe
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
market_data = pd.read_parquet('data/US/mkt_data.pq')
balance_sheet = pd.read_parquet('data/US/balance_sheets.pq')
GICS = pd.read_parquet('data/US/GICS.pq')
P = pd.read_parquet('data/US/universe_table.pq')
R = market_data['close'].unstack().reindex_like(P).pct_change()
market_data.dropna().sample(5)

,,datetime,open,high,low,close,adj_close,volume,MktCap
Date,Ticker,,,,,,,,
2009-08-13,X,2009-08-13,45.79,47.41,44.95,47.32,42.5706,14729570,6.065262e+09
2014-12-26,JBHT,2014-12-26,84.96,85.47,84.81,84.92,78.6516,558890,1.003815e+10
2022-09-28,MMI,2022-09-28,32.00,32.44,31.45,32.19,31.9397,274503,1.280455e+09
2012-01-18,EA,2012-01-18,17.72,18.28,17.70,18.26,18.0124,7761965,5.871940e+09
2022-07-25,WYNN,2022-07-25,61.86,62.36,60.05,62.23,62.2300,2178998,7.066295e+09


In [23]:
import pandas_ta as ta
from tqdm import tqdm
results ={}
for ticker in tqdm(market_data.index.get_level_values(1).unique()):
    df_ticker = market_data.xs(ticker,level=1)
    try :
        metrics = ta.squeeze(df_ticker['high'],df_ticker['low'],df_ticker['close'],
                    bb_std=2,
                    mom_length=5,
                    bb_length=5).iloc[:,0].fillna(0)
        results[ticker] = metrics

    except :
        pass

  3%|▎         | 25/866 [00:00<00:06, 121.46it/s]

Date
2008-01-02     36.7886
2008-01-03     36.3388
2008-01-04     35.8389
2008-01-07     35.8689
2008-01-08     35.7390
                ...   
2023-03-30    135.5400
2023-03-31    138.7300
2023-04-03    138.0900
2023-04-04    139.1100
2023-04-05    139.2700
Name: high, Length: 3842, dtype: float64
Date
2008-01-02    36.7800
2008-01-03    36.6700
2008-01-04    35.9300
2008-01-07    35.0100
2008-01-08    33.5501
               ...   
2023-03-30    42.8900
2023-03-31    42.7500
2023-04-03    42.9500
2023-04-04    41.5200
2023-04-05    40.1900
Name: high, Length: 3842, dtype: float64
Date
2008-01-02    14.58
2008-01-03    13.62
2008-01-04    12.70
2008-01-07    12.93
2008-01-08    12.25
              ...  
2023-03-30    14.70
2023-03-31    14.76
2023-04-03    14.62
2023-04-04    14.55
2023-04-05    14.14
Name: high, Length: 3842, dtype: float64
Date
2008-01-02     38.15
2008-01-03     38.16
2008-01-04     37.59
2008-01-07     37.42
2008-01-08     37.85
               ...  
2023-03-30    11

  4%|▍         | 38/866 [00:00<00:06, 123.16it/s]

Date
2008-01-02    21.59
2008-01-03    21.27
2008-01-04    21.35
2008-01-07    22.14
2008-01-08    22.48
              ...  
2023-03-30    23.96
2023-03-31    24.14
2023-04-03    24.21
2023-04-04    23.94
2023-04-05    24.24
Name: high, Length: 3842, dtype: float64
Date
2008-01-02    62.71
2008-01-03    63.87
2008-01-04    63.28
2008-01-07    64.00
2008-01-08    64.22
              ...  
2023-03-30    64.79
2023-03-31    64.67
2023-04-03    65.18
2023-04-04    65.10
2023-04-05    64.74
Name: high, Length: 3842, dtype: float64
Date
2021-04-15    31.6899
2021-04-16    33.9400
2021-04-19    32.6200
2021-04-20    34.1350
2021-04-21    33.5600
               ...   
2023-03-30    27.1750
2023-03-31    24.0200
2023-04-03    25.5800
2023-04-04    25.5350
2023-04-05    26.2300
Name: high, Length: 498, dtype: float64
Date
2008-01-02     27.35
2008-01-03     27.05
2008-01-04     26.65
2008-01-07     26.34
2008-01-08     26.70
               ...  
2020-05-04    190.46
2020-05-05    193.73
2020-05-

  9%|▉         | 78/866 [00:00<00:06, 124.78it/s]

...

Date
2020-09-24    36.000
2020-09-25    34.000
2020-09-28    35.000
2020-09-29    30.500
2020-09-30    31.000
               ...  
2023-03-30    26.890
2023-03-31    27.390
2023-04-03    27.450
2023-04-04    27.410
2023-04-05    26.285
Name: high, Length: 637, dtype: float64
Date
2009-10-07     28.97
2009-10-08     27.44
2009-10-09     27.49
2009-10-12     27.90
2009-10-13     27.26
               ...  
2023-03-30    191.90
2023-03-31    194.01
2023-04-03    192.08
2023-04-04    192.72
2023-04-05    193.34
Name: high, Length: 3397, dtype: float64
Date
2008-01-02     38.00
2008-01-03     37.83
2008-01-04     37.30
2008-01-07     35.66
2008-01-08     35.37
               ...  
2023-03-30    206.30
2023-03-31    211.66
2023-04-03    213.72
2023-04-04    213.96
2023-04-05    214.36
Name: high, Length: 3842, dtype: float64
Date
2008-01-02     23.51
2008-01-03     22.64
2008-01-04     22.20
2008-01-07     21.57
2008-01-08     22.18
               ...  
2023-03-30    315.56
2023-03-31    315.

 97%|█████████▋| 844/866 [00:07<00:00, 100.05it/s]

Date
2008-01-02     81.4500
2008-01-03     80.4900
2008-01-04     79.1100
2008-01-07     76.6500
2008-01-08     77.7100
                ...   
2023-03-30    130.6000
2023-03-31    132.1000
2023-04-03    132.6000
2023-04-04    131.8300
2023-04-05    130.1587
Name: high, Length: 3842, dtype: float64
Date
2008-01-02    13.1000
2008-01-03    12.7700
2008-01-04    12.3900
2008-01-07    12.4900
2008-01-08    12.4800
               ...   
2020-06-10     0.3403
2020-06-23     0.3403
2020-06-29     0.3403
2020-07-02     0.3403
2020-07-10     0.3403
Name: high, Length: 2885, dtype: float64
Date
2014-09-22    124.4800
2014-09-23    122.0000
2014-09-24    124.5800
2014-09-25    123.8700
2014-09-26    121.9700
2014-09-29    121.9200
2014-09-30    121.6600
2014-10-01    119.5800
2014-10-02    119.2100
2014-10-03    122.5300
2014-10-06    123.3500
2014-10-07    119.3900
2014-10-08    120.0100
2014-10-09    120.8900
2014-10-10    120.0100
2014-10-13    117.6600
2014-10-14    114.9150
2014-10-15    113

100%|██████████| 866/866 [00:08<00:00, 106.43it/s]

Date
2019-09-03    9.85
2019-09-04    9.82
2019-09-05    9.95
2019-09-06    9.85
2019-09-09    9.85
              ... 
2023-01-09    1.01
2023-01-10    1.07
2023-01-11    1.12
2023-01-12    1.15
2023-01-13    1.22
Name: high, Length: 849, dtype: float64
Date
2008-01-02     22.00
2008-01-03     21.18
2008-01-04     21.17
2008-01-07     20.78
2008-01-08     20.66
               ...  
2022-02-10    226.80
2022-02-11    218.95
2022-02-14    194.92
2022-02-15    194.92
2022-02-16    194.92
Name: high, Length: 3558, dtype: float64
Date
2008-01-02     94.55
2008-01-03     94.74
2008-01-04     94.36
2008-01-07     93.15
2008-01-08     92.05
               ...  
2023-03-30    109.57
2023-03-31    110.17
2023-04-03    116.85
2023-04-04    116.70
2023-04-05    117.00
Name: high, Length: 3842, dtype: float64
Date
2008-01-02    45.4000
2008-01-03    44.4000
2008-01-04    44.4700
2008-01-07    44.6500
2008-01-08    44.9999
               ...   
2023-03-30    38.6600
2023-03-31    39.3100
2023-04-03 

In [20]:
ticker = 'AAPL'
import pandas_ta as ta
df_ticker = market_data.xs(ticker,level=1)
ta.squeeze(df_ticker['high'],df_ticker['low'],df_ticker['close'],
           bb_std=2,
           mom_length=5,
           bb_length=5).iloc[:,0]

Date
2008-01-02    200.2588
2008-01-03    197.3888
2008-01-04    193.0012
2008-01-07    183.5988
2008-01-08    182.4592
                ...   
2023-03-30    162.4700
2023-03-31    165.0000
2023-04-03    166.2900
2023-04-04    166.8400
2023-04-05    165.0500
Name: high, Length: 3842, dtype: float64


Date
2008-01-02         NaN
2008-01-03         NaN
2008-01-04         NaN
2008-01-07         NaN
2008-01-08         NaN
                ...   
2023-03-30    2.325000
2023-03-31    2.586667
2023-04-03    3.026667
2023-04-04    4.210000
2023-04-05    4.980000
Name: SQZ_5_2.0_20_1.5, Length: 3842, dtype: float64

In [16]:
df_ticker['high']

Date
2008-01-02    200.2588
2008-01-03    197.3888
2008-01-04    193.0012
2008-01-07    183.5988
2008-01-08    182.4592
                ...   
2023-03-30    162.4700
2023-03-31    165.0000
2023-04-03    166.2900
2023-04-04    166.8400
2023-04-05    165.0500
Name: high, Length: 3842, dtype: float64